<a href="https://colab.research.google.com/github/MonteiroAlvesMatheus/Deteccao_de_veiculos/blob/main/Detecao_de_veiculo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import time

cap = cv2.VideoCapture('Teste_carros_av.avi')
#  while cap.isOpened():
#   ret, image = cap.read()

#    if not ret:
#      break

#    cv2_imshow(image) # Note cv2_imshow, not cv2.imshow

#    cv2.waitKey(1) & 0xff

#    cv2.destroyAllWindows()
#    cap.release()

path_label = 'coco.names'

classes =[]
with open (path_label, 'rt') as f:
  classes = f.read().rstrip('\n').split('\n')

  wght_hght_target = 416
  model_config = 'yolov3.cfg'
  model_weight = 'yolov3.weights'
  confThreshold = 0.4
  nmsThreshold = 0.2
  incount1 = 0
  incount2 = 0
  incount3 = 0
  incount4 = 0
  incount5 = 0
  incount_reset = 0
  start_time = time.time()

  net_yolo = cv2.dnn.readNetFromDarknet(model_config,model_weight)
  net_yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
  net_yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

result = cv2.VideoWriter('result.avi',cv2.VideoWriter_fourcc(*'XVID'),20,(640,480))

def findObject(outputs,img):
      heightTar, weightTar, channelsTar = img.shape
      bbox = []
      classId = []
      confidance = []
      count1 = 0
      count2 = 0
      count3 = 0
      count4 = 0

      for output in outputs:
        for det in output:
            score = det[5:]
            classIds = np.argmax(score)
            confids = score[classIds]
            if classIds == 2 or classIds == 3 or classIds == 7 or classIds == 5:
                if confids > confThreshold:
                    w, h = int (det[2]*weightTar), int (det[3]*heightTar)
                    x, y = int ((det[0]*weightTar)-w/2), int ((det[3]*heightTar)- h/2)
                    bbox.append([x,y,w,h])
                    classId.append(classIds)
                    confidance.append(float(confids))
                    if (int(img.shape[0] / 2) - 3) < y < (int(img.shape[0] / 2) + 3):
                        if classes == 2:
                            count1 = count1 + 1
                        if classes == 3:
                            count2 = count2 + 1
                        if classes == 5:
                            count3 = count3 + 1
                        if classes == 7:
                            count4 = count4 + 1  
            else:
                continue
             
      draw_box = cv2.dnn.NMSBoxes(bbox, confidance, confThreshold, nmsThreshold)
      for i in draw_box:
        i = i[0]
        box = bbox[i]
        x, y, w, h = box[0],box[1],box[2],box[3]
        cv2.rectangle(img,(x,y),(x+w, y+h), (255,255,0),2)
        cv2.putText(img,f'{classes[classId[i]].upper()} {int(confidance[i]*100)}%', (x , y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,0) , 2)  

        cv2.line (img, (0, int(img.shape[0]/2)+3), (int(img.shape[1]),int(img.shape[0]/2)+3), (0,0,100),1)  
        cv2.line (img, (0, int(img.shape[0]/2)-3), (int(img.shape[1]),int(img.shape[0]/2)-3), (0,0,100),1)

        return count1, count2, count3, count4

while True:
  success, img = cap.read()
  img = cv2.resize(img, (640,480))
  cv2_imshow(img)
  blob = cv2.dnn.blobFromImage(img, 1/255, (wght_hght_target,wght_hght_target),[0,0,0,0], 1, crop = False)
  net_yolo.setInput(blob)
  LayerNames = net_yolo.getLayerNames()
  outputNames = [LayerNames[i[0]-1] for i in net_yolo.getUnconnectedOutLayers()]
  outputs = net_yolo.forward (outputNames)
  counter1, counter2, counter3, counter4 = findObject(outputs,img)

  incount1 = incount1 + counter1
  incount2 = incount2 + counter2
  incount3 = incount3 + counter3
  incount4 = incount4 + counter4
  incount5 = incount5 + (counter1 + counter2 + counter3 + counter4)

  run_time = time.time()
  incount_reset = int (time.time()-start_time)
  if incount_reset == 3600:
    incount1 = 0
    incount2 = 0
    incount3 = 0
    incount4 = 0
    incount5 = 0
    incount_reset = 0
    start_time = run_time

  cv2.putText(img,f'counting Car: {incount1}', (25,20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,0,0) , 2)
  cv2.putText(img,f'counting Motorcycle: {incount2}', (25,20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,200,0) , 2)
  cv2.putText(img,f'counting Bus: {incount3}', (25,20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,200) , 2)
  cv2.putText(img,f'counting Truck: {incount4}', (25,20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,200,200) , 2)
  cv2.putText(img,f'Total: {incount5}', (25,20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,0) , 2)
  result.write(img)
  cv2_imshow(img)

  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
result.release()
cv2.destroyAllWindows()


FileNotFoundError: ignored